# **Τεχνητή Νοημοσύνη 2**
#**Εργασία 4α**
Στέφανος Μπακλαβάς 1115201700093


In [ ]:

from google.colab import drive
drive.mount("/content/drive")



import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


!pip install beautifulsoup4
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np
from datetime import datetime

from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import MultinomialNB



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score



Mounted at /content/drive


In [ ]:
! pip install datasets transformers
!pip install torchvision 
!pip install sentencepiece

import torch
import torchvision


from sklearn import preprocessing
from sklearn.metrics import f1_score
vaccine_train_set = r'/content/drive/MyDrive/Colab Notebooks/AI_2/AI_2_HW1/vaccine_train_set.csv'
vaccine_validation_set = r'/content/drive/MyDrive/Colab Notebooks/AI_2/AI_2_HW1/vaccine_validation_set.csv'

     |████████████████████████████████| 312 kB 4.9 MB/s 
     |████████████████████████████████| 3.8 MB 29.8 MB/s 
     |████████████████████████████████| 134 kB 47.1 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 42.7 MB/s 
     |████████████████████████████████| 1.1 MB 25.9 MB/s 
     |████████████████████████████████| 127 kB 35.7 MB/s 
     |████████████████████████████████| 596 kB 35.6 MB/s 
     |████████████████████████████████| 6.5 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 34.9 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 144 kB 46.2 MB/s 
     |████████████████████████████████| 271 kB 45.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,050 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


## Utilizing Gpu if available


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## Reading datasets

In [ ]:
df_train = pd.read_csv(vaccine_train_set)
df_validation = pd.read_csv(vaccine_validation_set)

#df_train = df_train[:100]
#df_validation = df_validation[:100]

In [ ]:
df_train

,Unnamed: 0,tweet,label
0,0,Sip N Shop Come thru right now #Marjais #Popul...,0
1,1,I don't know about you but My family and I wil...,1
2,2,@MSignorile Immunizations should be mandatory....,2
3,3,President Obama spoke in favor of vaccination ...,0
4,4,"""@myfoxla: Arizona monitoring hundreds for mea...",0
...,...,...,...
15971,15971,@Salon if u believe the anti-vax nutcases caus...,1
15972,15972,How do you feel about parents who don't #vacci...,0
15973,15973,70 Preschoolers Tested for Measles in Simi Val...,0
15974,15974,Finance Minister: Budget offers room to procur...,0


## Data Pre-processing


Παρακάτω γίνεται η εξής προεπεξεργασία στα δεδομένα:

1.   Απαλοιφή των σημείων στίξης και διάφορων άλλων χαρακτήρων
2.   Καθαρισμός των πολλών κενών διαστημάτων και αντικατάστασή τους με ένα κενό
3.   Μετατροπή όλων των γραμμάτων σε πεζά



Είχε δοκιμαστεί και stemming αλλά δεν λειτούργησε καλύτερα.Επίσης δοκιμάστηκε να αφαιρεθούν τα emogis όμως με την τρέχουσα προεπεξεργασία αφαιρούνται έτσι κι αλλιώς.Για το λόγο αυτό οι γραμμές αυτές είναι σχολιασμένες 


In [ ]:
import re



df_train['tweet'] = df_train['tweet'].str.translate({ord(i): " " for i in ',–#%$&*()@''-‘"":.!?/'})
df_train.columns = df_train.columns.str.replace('  ', ' ')
df_train['tweet'] = df_train['tweet'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
df_train['tweet'] = df_train['tweet'].str.lower()
#df_train['tweet'] = df_train['tweet'].apply(lambda x: [stemmer.stem(y) for y in x])
#df_train['tweet'] = df_train['tweet'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
#df_train['tweet'] = df_train['tweet'].str.replace('\d+', '')


df_validation['tweet'] = df_validation['tweet'].str.translate({ord(i): " " for i in ',–#%$&*()@''-‘"":.!?/'})
df_validation.columns = df_validation.columns.str.replace('  ', ' ')
df_validation['tweet'] = df_validation['tweet'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
df_validation['tweet'] = df_validation['tweet'].str.lower()
#df_validation['tweet'] = df_validation['tweet'].apply(lambda x: [stemmer.stem(y) for y in x])
#df_validation['tweet'] = df_validation['tweet'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
#df_validation['tweet'] = df_validation['tweet'].str.replace('\d+', '')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()


Χρησιμοποιείται ο distilbert-base-uncased autotokenizer

In [ ]:
from transformers.models.auto.tokenization_auto import AutoTokenizer
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased',do_lower_case=True)
#assert isinstance(tokenizer,PreTrainedTokenizerFast)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Ξεχώρισμα των tweets και των labels από κάθε dataset για training και validation

In [ ]:
sentences = df_train['tweet'].values
labels = df_train['label'].values
#train_dataset = np.array([[sentences],[labels]]).transpose()
train_dataset = list(zip(*(sentences,labels)))


sentences = df_validation['tweet'].values
labels = df_validation['label'].values
validation_dataset = list(zip(*(sentences,labels)))

print(train_dataset[0][1])
print(len(train_dataset))


0
15976


In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:   user they had massive surge in with covid deaths immediately after the vaccine rollout way more than they had during the entirety of the pandemic up to that point fuck those jabs dr death 
Tokenized:  ['user', 'they', 'had', 'massive', 'surge', 'in', 'with', 'co', '##vid', 'deaths', 'immediately', 'after', 'the', 'vaccine', 'roll', '##out', 'way', 'more', 'than', 'they', 'had', 'during', 'the', 'entirety', 'of', 'the', 'pan', '##de', '##mic', 'up', 'to', 'that', 'point', 'fuck', 'those', 'ja', '##bs', 'dr', 'death']
Token IDs:  [5310, 2027, 2018, 5294, 12058, 1999, 2007, 2522, 17258, 6677, 3202, 2044, 1996, 17404, 4897, 5833, 2126, 2062, 2084, 2027, 2018, 2076, 1996, 15700, 1997, 1996, 6090, 3207, 7712, 2039, 2000, 2008, 2391, 6616, 2216, 14855, 5910, 2852, 2331]


Xρησιμοποιείται το collate_fn ορισμα του dataloader ώστε να γίνει tokenization και padding των tweets σε batches. To batch size που χρησιμοποιήθηκε είναι το 16 αφού δεν χώραγε μεγαλύτερο στην RAM του colab.

In [ ]:

def pad_collate_fn(batch):

  sentences = []
  labels = []
  for row in batch:
    sentences.append(row[0])
    labels.append(row[1])
  
  input_ids = []
  attention_masks = []
  
  max_length = 0
  for sent in sentences:
    length = len(sent)
    if length > max_length:
      max_length = length


  if max_length > 512:
    max_length = 512

  #print("max_length = ",max_length)  
  for sent in sentences:
        # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_length,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  return input_ids,attention_masks,labels



Χρησιμοποιείται ως model το distilbert-base-uncased του AutoModelForSequenceClassification , ενώ το learning rate με τα καλύτερα αποτελέσματα ήταν το 5e-5. Πρέπει να σημειωθεί ότι και τα τρια learning rates που χρησιμοποιήθηκαν είχαν σχεδόν ίδιο αποτέλεσμα.Συγκεκριμένα έγιναν δοκιμές με lr = 2e-5, lr = 3e-5 και lr = 5e-5.


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                           num_labels = 3,
                                           output_attentions = False,
                                           output_hidden_states = False)

# Tell pytorch to run this model on the GPU.
optimizer = torch.optim.Adam(model.parameters(),  lr = 5e-5, 
                  eps = 1e-8 
                )


#load model to cuda device
model.cuda()



dataloader_train = DataLoader(
            train_dataset,  # The training samples.
            shuffle = True, 
            batch_size = 16, # Trains with this batch size.
            collate_fn=pad_collate_fn
        )

dataloader_validation = DataLoader(
            validation_dataset, # The validation samples.
            shuffle = True, 
            batch_size = 16, # Evaluate with this batch size.
            collate_fn=pad_collate_fn
        )

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

Training και Validation του μοντελου.

In [ ]:
from numpy.core.fromnumeric import argmax
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

from scipy.special import softmax
import random 
import warnings
warnings.filterwarnings('ignore')

# Set the seed value.
seed_val = 5

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


def Average(lst):
    return sum(lst) / len(lst)

train_loss = []
validation_loss = []
epochs = []
fpr = dict()
tpr = dict()
roc_auc = dict()
best_epoch_preds = []
best_epoch_y_values = []
prev_max_presicion = 0

for epoch in range(4):
  
  batch_losses_train = []
  batch_losses_validation = []

  preds = []
  y_values = []

  model.train()

  for batch in dataloader_train:


      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      model.zero_grad()        

      output = model(b_input_ids,  
                             attention_mask=b_input_mask, 
                             labels=b_labels)

      loss = output.loss
      y_pred = output.logits
      batch_losses_train.append(loss)
     
    
      #Perform backpropagation starting from the loss calculated in this epoch
      loss.backward()

      
      #Update model's weights based on the gradients calculated during backprop
      optimizer.step()


  train_loss.append(Average(batch_losses_train))
  batch_losses_train.clear

  model.eval()

  for batch in dataloader_validation:
   

      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      labels = batch[2].to(device)


      with torch.no_grad():        

          output = model(b_input_ids,  
                             attention_mask=b_input_mask, 
                             labels=labels)


      loss = output.loss
      y_pred = output.logits
      batch_losses_validation.append(loss)
     
      b_labels = labels.to('cpu').numpy()
      y_pred = y_pred.detach().cpu().numpy()

      y_pred = softmax(y_pred)
      for pred in y_pred:
        preds.append(argmax(pred))
      for val in b_labels:
        y_values.append(val)   

      batch_losses_validation.append(loss.item())




  validation_loss.append(Average(batch_losses_validation))
  batch_losses_validation.clear

  #############################################################################################################
  #calculating scores
  epochs.append(epoch)
  validation_loss.append(Average(batch_losses_validation))
 

  #print("y_values length = ",len(y_values))
  f1 = f1_score(y_values,preds,average='weighted',labels=np.unique(preds))
  print('F1 score: ',f1)
  precision = precision_score(y_values,preds, average='weighted',labels=np.unique(preds))
  print('Precision score: ',precision)


  if precision > prev_max_presicion:
    prev_max_presicion = precision
    prev_max_f1 = f1
    best_epoch_preds = preds.copy()
    best_epoch_y_values = y_values.copy()

  
  output = set(y_values) - set(preds)
  print(output)
  preds.clear()
  y_values.clear()
  batch_losses_validation.clear


best_epoch_y_values = label_binarize(best_epoch_y_values,classes = [0,1,2])
best_epoch_preds = label_binarize(best_epoch_preds,classes = [0,1,2])

#for val in best_epoch_y_values:
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(best_epoch_y_values[:, i], best_epoch_preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(best_epoch_y_values.ravel(), best_epoch_preds.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

print(prev_max_f1)
print(prev_max_presicion)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


F1 score:  0.7410777419594897
Precision score:  0.7422840939089211
set()
F1 score:  0.7490608992834176
Precision score:  0.7494403608176932
set()
F1 score:  0.7469718301429763
Precision score:  0.7480283724485197
set()
F1 score:  0.7328176384748246
Precision score:  0.7383608844186436
set()
0.7490608992834176
0.7494403608176932
